In [1]:
# 引入所需库
import numpy as np
import pandas as pd
import time
import copy

In [2]:
# 读取文件及数据初始化
pt_tmp=pd.read_excel("JSP_dataset.xlsx",sheet_name="Processing Time",index_col =[0])
ms_tmp=pd.read_excel("JSP_dataset.xlsx",sheet_name="Machines Sequence",index_col =[0])

dfshape=pt_tmp.shape
num_mc=dfshape[1] # 机器台数
num_job=dfshape[0] # 工序数
num_gene=num_mc*num_job # 单条染色体中的基因数

pt=[list(map(int, pt_tmp.iloc[i])) for i in range(num_job)]
ms=[list(map(int,ms_tmp.iloc[i])) for i in range(num_job)]



In [4]:
# 设定初始参数

POPULATION_SIZE = 30 # 种群大小
CROSSOVER_RATE = 0.8 # 交叉概率
MUTATION_RATE = 0.2 # 变异概率
MUTATION_SELECTION_RATE = 0.2 # 变异选择率
NUM_MUTATION_JOBS = round(num_gene*MUTATION_SELECTION_RATE) # 变异基因数
NUM_ITERATION = 2000 # 迭代次数
ckeck_time = False # 是否计算运行时间

In [5]:
# 开始计时
if ckeck_time:
    start_time = time.time()

In [6]:
# 生成初始种群
Tbest=999999999999999 # 用于记录最优解
best_list,best_obj=[],[] # 用于记录最优解的染色体和目标函数值
population_list=[] # 用于存储种群
for i in range(POPULATION_SIZE):
    nxm_random_num=list(np.random.permutation(num_gene)) # 生成一个随机序列，from 0 to num_gene-1   
    population_list.append(nxm_random_num) # 添加到种群中
    for j in range(num_gene):
        population_list[i][j]=population_list[i][j]%num_job # 生成随机基因序列

In [7]:
# 生成新一代
for n in range(NUM_ITERATION):
    Tbest_now=99999999999           
   
    # 交配
    parent_list=copy.deepcopy(population_list)      # 深拷贝父母染色体
    offspring_list=copy.deepcopy(population_list)   # 深拷贝子代染色体
    S=list(np.random.permutation(POPULATION_SIZE))  # 返回随机的序列，用于选择交配父母染色体
    
    for m in range(int(POPULATION_SIZE/2)):         # 交配次数
        crossover_prob=np.random.rand()             # 生成随机交叉概率
        if CROSSOVER_RATE>=crossover_prob:          # 交叉
            parent_1= population_list[S[2*m]][:]    # 相邻两个进行交叉
            parent_2= population_list[S[2*m+1]][:]
            child_1=parent_1[:]
            child_2=parent_2[:]
            cutpoint=list(np.random.choice(num_gene, 2, replace=False)) # 生成两个随机的交叉点
            cutpoint.sort() # 交叉点排序
        
            child_1[cutpoint[0]:cutpoint[1]]=parent_2[cutpoint[0]:cutpoint[1]]
            child_2[cutpoint[0]:cutpoint[1]]=parent_1[cutpoint[0]:cutpoint[1]]
            offspring_list[S[2*m]]=child_1[:]
            offspring_list[S[2*m+1]]=child_2[:]
    
    # 修复，确保每个工序在每台机器上只出现一次
    for m in range(POPULATION_SIZE):
        job_count={} # 用于记录每个数字出现的次数和第一次出现的位置
        larger,less=[],[] # 'larger'记录超过正常次数的染色体，less记录小于正常次数的染色体
        for i in range(num_job):
            if i in offspring_list[m]:
                count=offspring_list[m].count(i) # 计算染色体中每个数字出现的次数
                pos=offspring_list[m].index(i) # 记录第一次出现的位置
                job_count[i]=[count,pos] # 将以上两个值存储到job_count字典中
            else:
                count=0
                job_count[i]=[count,0]
            if count>num_mc:
                larger.append(i)
            elif count<num_mc:
                less.append(i)
                
        for k in range(len(larger)):
            chg_job=larger[k] # 需要处理的数字
            while job_count[chg_job][0]>num_mc:
                for d in range(len(less)):
                    if job_count[less[d]][0]<num_mc:                    
                        offspring_list[m][job_count[chg_job][1]]=less[d] # 将超过正常次数的数字替换为小于正常次数的数字
                        job_count[chg_job][1]=offspring_list[m].index(chg_job) # 更新第一次出现的位置
                        job_count[chg_job][0]=job_count[chg_job][0]-1 # 更新出现次数
                        job_count[less[d]][0]=job_count[less[d]][0]+1 # 更新出现次数                   
                    if job_count[chg_job][0]==num_mc: # 如果处理完毕，则跳出循环
                        break     

    # 变异
    """循环变异，选择一个变异位点，将该位点的值赋给下一个变异位点，循环往复，直至最后一个变异位点的值赋给第一个变异位点"""""
    for m in range(len(offspring_list)):
        mutation_prob=np.random.rand() # 生成随机变异概率
        if MUTATION_RATE >= mutation_prob:
            m_chg=list(np.random.choice(num_gene, NUM_MUTATION_JOBS, replace=False)) # 选择变异的位点
            t_value_last=offspring_list[m][m_chg[0]] # 记录第一个变异位点的值
            for i in range(NUM_MUTATION_JOBS-1):
                offspring_list[m][m_chg[i]]=offspring_list[m][m_chg[i+1]] # 展示变异
            
            offspring_list[m][m_chg[NUM_MUTATION_JOBS-1]]=t_value_last # 将第一个变异位点的值赋给最后一个变异位点

    # 计算适应度
    """计算新种群的适应度
    由于需要利用轮盘法，所以需要计算每个染色体的适应度（倒数，用于轮盘）
    直接适用度用于比较，适应度越小，越优秀
    """
    total_chromosome=copy.deepcopy(parent_list)+copy.deepcopy(offspring_list) # 将亲代和子代染色体合并
    chrom_fitness,chrom_fit=[],[] # 用于存储每个染色体的倒数适应度（用于轮盘）和适应度
    total_fitness=0
    for m in range(POPULATION_SIZE*2):
        j_keys=[j for j in range(num_job)]
        key_count={key:0 for key in j_keys}
        j_count={key:0 for key in j_keys}
        m_keys=[j+1 for j in range(num_mc)]
        m_count={key:0 for key in m_keys}
        
        for i in total_chromosome[m]:
            gen_t=int(pt[i][key_count[i]])
            gen_m=int(ms[i][key_count[i]])
            j_count[i]=j_count[i]+gen_t
            m_count[gen_m]=m_count[gen_m]+gen_t
            
            if m_count[gen_m]<j_count[i]:
                m_count[gen_m]=j_count[i]
            elif m_count[gen_m]>j_count[i]:
                j_count[i]=m_count[gen_m]
            
            key_count[i]=key_count[i]+1
    
        makespan=max(j_count.values())
        chrom_fitness.append(1/makespan)
        chrom_fit.append(makespan)
        total_fitness=total_fitness+chrom_fitness[m]

KeyboardInterrupt: 